# Loading Libraries

In [ ]:
#Import Code
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import data
data = pd.read_csv('/content/drive/MyDrive/Week 4 Discovery/ohlc_data.csv')
tickers = data.ticker.unique()

In [ ]:
pip install yfinance

# Loading Dataset

In [ ]:
import yfinance as yf
import numpy as np
from functools import reduce
def get_log_return(list_of_tickers, start_date, end_date, interval="1d"): 
    stocks = list()
    for ticker in list_of_tickers:
        ticker_data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
        data={ticker: np.log(ticker_data['Adj Close']) - np.log(ticker_data['Adj Close'].shift(1))}
        ticker_log_return = pd.DataFrame(data=data)
        stocks.append(ticker_log_return)
    combined = reduce(lambda df1,df2: pd.merge(df1,df2,on='Date'), stocks)
    return combined
df= get_log_return(tickers, "2001-01-02", "2020-12-31")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

# Model

In [ ]:
data = df

In [ ]:
#loop  RUN AT OWN RISK 7HOUR LOAD TIME
def prediction(df):
  train_data, test_data = df[0:int(len(df)*0.7)], df[int(len(df)*0.7):]
  training_data = pd.DataFrame({'returns':train_data.values})
  test_data = pd.DataFrame({'returns':test_data.values})
  history = train_data.values.tolist()
  model_predictions = []
  N_test_observations = len(test_data)
  for time_point in range(N_test_observations):
      model = ARIMA(history, order=(4,1,0))
      model_fit = model.fit()
      output = model_fit.forecast()
      yhat = output[0]
      model_predictions.append(yhat)
      true_test_value = test_data.loc[time_point]['returns']
      history.append(true_test_value)
  MSE_error = mean_squared_error(test_data, model_predictions)
  return MSE_error

# Results

In [2]:
result = pd.DataFrame()
dictionary = {}
arr =[]
for ticker in tickers:
  df = data[ticker][1:]
  MSE = prediction(df)
  arr.append(MSE)
  result[ticker] = MSE
  dictionary[ticker] =  MSE

In [1]:
 #MSE
MSE = pd.DataFrame(dictionary,index=[0])
result = pd.melt(MSE).rename(columns ={'variable' :'tickers', 'value':'MSE'})
result['RMSE']= np.sqrt((result['MSE']))
result